# 城市设计分析技术Python自学教程14

### 面向对象编程03-特殊方法与继承

##### [计算城市设计实验室(Computational Urban Design Lab)](https://www.tjcud.cn/)

#### 两种类型测试(type, isinstance)

In [ ]:
class A(object): pass
a = A()
print(type(a))           # A (实际上打印出 < class '__main__.A' >)
print(type(a) == A)      # True
print(isinstance(a, A))  # True

#### 特殊方法
#### 全等测试 (__eq__)
#### 问题来了：是否 a1 == a2?

In [ ]:
class A(object):
    def __init__(self, x):
        self.x = x
a1 = A(5)
a2 = A(5)
print(a1 == a2)  # False!

#### 一个能部分解决问题的方案: __eq__
__eq__ 方法告诉python如何去判断两个值是否全等

In [ ]:
class A(object):
    def __init__(self, x):
        self.x = x
    def __eq__(self, other):
        return (self.x == other.x)
a1 = A(5)
a2 = A(5)
print(a1 == a2)  # True
print(a1 == 99)  # 报错！

#### 一个更好的方案:
这样就不会因为奇奇怪怪的原因而报错了:

In [ ]:
class A(object):
    def __init__(self, x):
        self.x = x
    def __eq__(self, other):
        return (isinstance(other, A) and (self.x == other.x))
a1 = A(5)
a2 = A(5)
print(a1 == a2)  # True
print(a1 == 99)  # False 

#### 转换为字符串（__str__ 和 __repr__）
#### 问题：
就像 == 一样，Python 并不真正知道如何打印我们的对象......

In [ ]:
class A(object):
    def __init__(self, x):
        self.x = x
a = A(5)
print(a) # 打印出 <__main__.A object at 0x102916128> (yuck!)

#### 一个能部分解决问题的方案：__str__

__str__ 方法告诉 Python 如何将对象转换为字符串，但在某些情况下（例如对象在列表中时）不使用它：

In [ ]:
class A(object):
    def __init__(self, x):
        self.x = x
    def __str__(self):
        return f'A(x={self.x})'
a = A(5)
print(a) # 打印出 A(x=5) 
print([a]) # 打印出 [<__main__.A object at 0x102136278>] 

#### 更好的解决方案：__repr__
__repr__ 方法用于列表（和其他地方）：

In [ ]:
# 注意：repr 应该是一种计算机可读的形式，比如 (eval(repr(obj)) == obj)，但我们不会那样使用它。
# 所以这是repr的简化使用。

class A(object):
    def __init__(self, x):
        self.x = x
    def __repr__(self):
        return f'A(x={self.x})'
a = A(5)
print(a) # 打印出 A(x=5) (better)
print([a]) # [A(x=5)]

#### 类级功能
#### 类属性
类属性是在类中指定的值，该值由该类的所有实例共享！
我们可以从该类的任何实例访问类属性，但是在任何地方更改这些值都会为每个实例更改它们。

In [ ]:
class A(object):
    dirs = ["up", "down", "left", "right"]

# 通常通过类直接访问类属性（没有实例！）
print(A.dirs) # ['up', 'down', 'left', 'right']

# 也可以通过实例访问：
a = A()
print(a.dirs)

# 但所有实例只有一个共享值：
a1 = A()
a1.dirs.pop() # 不是个好主意
a2 = A()
print(a2.dirs) # ['up', 'down', 'left'] ('right' 不见了)

#### 静态方法
可以直接调用类中的静态方法，而不必创建和/或引用特定对象。

In [ ]:
class A(object):
    @staticmethod
    def f(x):
        return 10*x

print(A.f(42)) # 420 (调用 A.f 而不创建 A 的实例)

#### 纸牌演示
您不需要掌握以下示例中的 __hash__ 方法：

In [ ]:
import random

class PlayingCard(object):
    numberNames = [None, "Ace", "2", "3", "4", "5", "6", "7",
                   "8", "9", "10", "Jack", "Queen", "King"]
    suitNames = ["Clubs", "Diamonds", "Hearts", "Spades"]
    CLUBS = 0
    DIAMONDS = 1
    HEARTS = 2
    SPADES = 3

    @staticmethod
    def getDeck(shuffled=True):
        deck = [ ]
        for number in range(1, 14):
            for suit in range(4):
                deck.append(PlayingCard(number, suit))
        if (shuffled):
            random.shuffle(deck)
        return deck

    def __init__(self, number, suit):
        
# Ace 的数字是 1，2...10，        
# 11 为杰克，12 为皇后，13 为国王        
# 花色为 0 代表俱乐部，1 代表钻石，        
# 2 红心，3 黑桃

        self.number = number
        self.suit = suit

    def __repr__(self):
        number = PlayingCard.numberNames[self.number]
        suit = PlayingCard.suitNames[self.suit]
        return f'<{number} of {suit}>'

    def getHashables(self):
        return (self.number, self.suit) # 返回可被哈希算法使用的元组

    def __hash__(self):
        # 你不用看懂这里
        return hash(self.getHashables())

    def __eq__(self, other):
        return (isinstance(other, PlayingCard) and
                (self.number == other.number) and
                (self.suit == other.suit))

# Show this code in action
print("PlayingCard 的演示将不断创建新的牌组，并且")
print("抽第一张牌，直到我们看到同一张牌两次.")
print()
cardsSeen = set()
diamondsCount = 0
 
# 现在继续抽卡，直到我们得到重复的卡片
while True:
    deck = PlayingCard.getDeck()
    drawnCard = deck[0]
    if (drawnCard.suit == PlayingCard.DIAMONDS):
        diamondsCount += 1
    print("  drawnCard:", drawnCard)
    if (drawnCard in cardsSeen): break
    cardsSeen.add(drawnCard)

# 然后报告我们抽了多少张牌
print("共计抽了:", 1+len(cardsSeen),"张牌")
print("共抽了:", diamondsCount,"张方片")

#### 继承
子类继承其超类的所有方法，然后可以添加或修改方法。
#### 指定超类

In [ ]:
class A(object):
    def __init__(self, x):
        self.x = x
    def f(self):
        return 10*self.x

class B(A):
    def g(self):
        return 1000*self.x

print(A(5).f()) # 50
print(B(7).g()) # 7000
print(B(7).f()) # 70 (B 类继承了 A 类的方法 f)
print(A(5).g()) # 报错 (A 类没有方法 g)

#### 覆盖方法
我们可以通过覆盖它来更改子类中方法的行为。

#### isinstance 和 type 在子类中的使用

In [ ]:
class A(object): pass
class B(A): pass
a = A()
b = B()
print(type(a) == A) # True
print(type(b) == A) # False
print(type(a) == B) # False
print(type(b) == B) # True
print()
print(isinstance(a, A)) # True
print(isinstance(b, A)) # True (奇怪吧~)
print(isinstance(a, B)) # False
print(isinstance(b, B)) # True

#### 怪兽（demo）

In [ ]:
# 这是你的基础类
class Monster(object):
    def __init__(self, strength, defense):
        self.strength = strength
        self.defense = defense
        self.health = 10

    def attack(self): # 返回造成的伤害值
        if self.health > 0:
            return self.strength

    def defend(self, damage): # 对自己造成伤害
        self.health -= damage

class MagicMonster(Monster):
    def __init__(self, strength, defense):
        super().__init__(strength, defense) # 大部分属性相同
        self.health = 5 # 开始的时候血量少一些

    def heal(self): # 只有魔法怪兽可以治疗自己
        if 0 < self.health < 5:
            self.health += 1

class NecroMonster(Monster):
    def attack(self): # NecroMonsters 可以在被干掉后继续攻击
        return self.strength

#### A额外阅读
有关这些主题的更多信息以及许多其他 OOP 相关主题，请查看以下链接：
     https://docs.python.org/3/tutorial/classes.html
     https://docs.python.org/3/reference/datamodel.html